In [1]:
import numpy as np
#for some reason, matplotlib crashes without these lines
a = np.zeros((5,5))
a@a
import matplotlib.pyplot as plt
from utils import get_mesh_vtk
from utils import get_data
from utils import strip_cross, reconstruct_cross
from KPCA import *
import Autoencoder
# from torchsummary import summary
# from Autoencoder import Autoencoder
import torch
import torch.nn as nn
from utils import calc_energy
import os
# from importlib import reload

In [2]:
device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)

In [ ]:

## Data reading
parameters = [
        (0.01,2.5,0.11111),
        (0.01,3.5,0.11111),
        (0.01,5.0,0.33333),
        (0.01,7.0,0.11111),
        (0.01,7.5,0.33333),
]
# Parameters:
# Re = 1
# Wi = 7.5
# beta = 0.6
Re ,Wi ,beta = parameters[0]
alpha = (1-beta)/(Re*Wi)

dx = 1/2**6
# type of simulation
case = 'cross'
#read file
X, Xmean = get_data(Re,Wi,beta, case, n_data= -2, dir_path='../npz_data')
print('Data shape: ',X.shape)
# # Reshape for pytorch NN
# X_torch = np2torch(X)
# print('Pytorch Input shape: ',X_torch.shape)



# #normalize data inside autoencoder
# lower_bound = (X_torch.min(dim = 0, keepdim=True)[0].min(dim=2, keepdim=True)[0]).float().to(device)
# upper_bound = (X_torch.max(dim = 0, keepdim=True)[0].max(dim=2, keepdim=True)[0]).float().to(device)